In [39]:
!nvidia-smi

Sun Aug 14 15:39:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P8    19W / 210W |    972MiB /  8192MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 資料讀取與前處理

In [57]:
###### 資料讀取 實際測試時此區改為讀取攔截之封包資料
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

csv_files = []
for dirname, _, filenames in os.walk('MachineLearningCSV/MachineLearningCVE'):
    for filename in filenames:
        csv_file = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))
        csv_files.append(csv_file)

# df = pd.concat([pd.read_csv(file) for file in csv_files[:8]], ignore_index=True)
df = pd.read_csv("MachineLearningCSV/MachineLearningCVE/week_2.csv",index_col = 0)

df.columns = df.columns.str.strip()
print("original length of df:", len(df))
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
print("after droping null values, the length of df:", len(df))

MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Tuesday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Wednesday-workingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\week.csv
MachineLearningCSV/MachineLearningCVE\week_2.csv
original length of df: 2830743
after droping null values, the length of df: 2827876


In [58]:
###### 針對標籤之處理，實際測試攔截封包資料時可拿掉
df_experiment = df.copy()
df_experiment.Label.replace("Web.*", "Web Attack", regex=True, inplace=True)
df_experiment.Label.replace(r'.*Patator$', "Brute Force", regex=True,inplace=True)
df_experiment.Label.replace(["DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris"], "DoS", inplace=True)

class_attack = ['PortScan', 'Web Attack', 'Brute Force', 'DDoS', 'Bot','Infiltration', 'DoS', 'Heartbleed']
df_experiment.Label.replace(class_attack, value='attack', inplace=True)
df_experiment.Label.value_counts()

BENIGN    2271320
attack     556556
Name: Label, dtype: int64

In [59]:
# data split and preprocess
from sklearn.model_selection import train_test_split
import joblib

std_scaler,mm_scaler,le = joblib.load("model/std_mm_le_new.save")

y = df_experiment.Label
X = df_experiment.drop(columns='Label')
labels = y.unique()

X = std_scaler.transform(X)

classes = y.nunique()
print(X.shape)
print("number of labels:", classes)
print("instances per label\n", y.value_counts())
print("labels:", labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

print("after spliting the data:\n")
print("training data length:", len(X_train))
print("test data length:", len(X_test))

X_train = mm_scaler.transform(X_train)
X_test = mm_scaler.transform(X_test)

print("after MinMaxScaler")
print(X_train.shape)

#標籤編碼 可不跑
# y_train = le.transform(y_train)
# y_test = le.transform(y_test)

(2827876, 77)
number of labels: 2
instances per label
 BENIGN    2271320
attack     556556
Name: Label, dtype: int64
labels: ['BENIGN' 'attack']
after spliting the data:

training data length: 2120907
test data length: 706969
after MinMaxScaler
(2120907, 77)


In [60]:
###### Random Forest
import joblib

rng = np.random.RandomState(42)

model = joblib.load("model/pkl/new_randomForest_32.pkl")
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(y_train_pred)
print(y_train_pred.shape)
print(y_test_pred)
print(y_test_pred.shape)

print("percentage of Anomaly:", (list(y_test_pred).count(1)/y_test_pred.shape[0])*100)
print("percentage of Legit:",(list(y_test_pred).count(0)/y_test_pred.shape[0])*100)

[0 0 0 ... 1 0 1]
(2120907,)
[0 0 1 ... 0 0 0]
(706969,)
percentage of Anomaly: 19.7059559895837
percentage of Legit: 80.2940440104163


In [14]:
y_train

2002012    BENIGN
1915100    BENIGN
1293022    BENIGN
1300105    BENIGN
2639033    BENIGN
            ...  
2479407    BENIGN
1365503    BENIGN
413800     attack
1147176    BENIGN
2226647    attack
Name: Label, Length: 2120907, dtype: object

In [17]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_train, y_train_pred)
tn, fp, fn, tp = cf_matrix.ravel()
cf_matrix

array([[1702181,    1309],
       [    596,  416821]], dtype=int64)

# Test for CICFLOWMETER Data

In [51]:
###### 資料讀取 實際測試時此區改為讀取攔截之封包資料
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

df = pd.read_csv("MachineLearningCSV/testflows.csv")

df.columns = df.columns.str.strip()
print("original length of df:", len(df))
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
print("after droping null values, the length of df:", len(df))

original length of df: 20
after droping null values, the length of df: 20


In [53]:
df = df.drop(["src_ip" , "dst_ip", "src_port", "src_mac", "dst_mac", "protocol", "timestamp"], axis = 1)

In [54]:
df

,dst_port,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
0,53,13318.538666,5.345932e+04,600.666500,300.333250,300.333250,4,4,340,372,...,4.0,8.830114e+05,2.528780e+04,85.0,93.0,0,4,4,340,372
1,53,9852.409363,7.795048e+04,811.984125,405.992063,405.992063,4,4,340,428,...,4.0,5.304561e+06,4.187700e+04,85.0,107.0,0,4,4,340,428
2,53,9401.321411,8.254159e+04,850.944208,425.472104,425.472104,4,4,344,432,...,4.0,5.934541e+04,5.005408e+04,86.0,108.0,0,4,4,344,432
3,53,9042.263031,9.555130e+04,884.734272,442.367136,442.367136,4,4,388,476,...,4.0,2.650048e+05,6.451164e+04,97.0,119.0,0,4,4,388,476
4,53,4065.513611,1.889060e+05,1967.771053,983.885527,983.885527,4,4,340,428,...,0.0,0.000000e+00,0.000000e+00,85.0,107.0,0,4,4,340,428
5,53,10650.634766,1.464701e+05,751.128940,375.564470,375.564470,4,4,344,1216,...,4.0,5.064125e+04,2.588101e+05,86.0,304.0,0,4,4,344,1216
6,53,11152.744293,7.316585e+04,717.312241,358.656120,358.656120,4,4,364,452,...,4.0,3.914684e+07,4.422597e+04,91.0,113.0,0,4,4,364,452
7,53,8101.940155,1.155279e+05,987.417809,493.708905,493.708905,4,4,352,584,...,4.0,7.015927e+07,1.253470e+05,88.0,146.0,0,4,4,352,584
8,53,7579.803467,1.329850e+05,1055.436336,527.718168,527.718168,4,4,328,680,...,4.0,1.331525e+06,1.613436e+06,82.0,170.0,0,4,4,328,680
9,53,34064.292908,2.759488e+04,234.850024,117.425012,117.425012,4,4,336,604,...,4.0,6.935463e+05,1.639796e+04,84.0,151.0,0,4,4,336,604


In [55]:
# data split and preprocess
from sklearn.model_selection import train_test_split
import joblib

std_scaler,mm_scaler,le = joblib.load("model/std_mm_le_new.save")

X = df

X = std_scaler.transform(X)

print("after StandardScaler")
print(X.shape)

X = mm_scaler.transform(X)

print("after MinMaxScaler")
print(X.shape)

#標籤編碼 可不跑
# y_train = le.transform(y_train)
# y_test = le.transform(y_test)

after StandardScaler
(20, 77)
after MinMaxScaler
(20, 77)


In [56]:
###### Random Forest
import joblib

rng = np.random.RandomState(42)

model = joblib.load("model/pkl/new_randomForest_32.pkl")
X_pred = model.predict(X)

print(X_pred)
print(X_pred.shape)

print("percentage of Anomaly:", (list(X_pred).count(1)/X_pred.shape[0])*100)
print("percentage of Legit:",(list(X_pred).count(0)/X_pred.shape[0])*100)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(20,)
percentage of Anomaly: 0.0
percentage of Legit: 100.0
